# [Lesson 4: Persistence and Streaming](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/5/persistence-and-streaming)

In [35]:
from dotenv import load_dotenv

_ = load_dotenv()

In [36]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [37]:
tool = TavilySearchResults(max_results=2)

In [38]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [39]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = 'checkpoints.db'
conn = sqlite3.connect(db_path, check_same_thread=False)

memory = SqliteSaver(conn)

In [40]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [41]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [42]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [43]:
thread = {"configurable": {"thread_id": "1"}}

In [44]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='The current weather in San Francisco is partly cloudy, with a temperature of 13.3°C (55.9°F). The wind is blowing from the east-northeast at 4.5 mph (7.2 kph), and the humidity level is 30%. The visibility is 16 kilometers (9 miles), and there is no precipitation.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 1854, 'total_tokens': 1927, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1792}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-e11d88de-49b5-4189-8fe4-228739475286-0', usage_metadata={'input_tokens': 1854, 'output_tokens': 73, 'total_tokens': 1927, 'input_token_details': {'audio': 0, 'cache_read': 1792}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]


In [45]:
messages = [HumanMessage(content="What about is la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='The current weather in Los Angeles is clear, with a temperature of 14.4°C (57.9°F). The wind is coming from the northwest at 4.0 mph (6.5 kph), and the humidity level is 33%. There is no cloud cover, and the visibility is 16 kilometers (9 miles).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 1939, 'total_tokens': 2010, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1792}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-09ae0d47-c8b1-4026-8daf-dc4463711ac1-0', usage_metadata={'input_tokens': 1939, 'output_tokens': 71, 'total_tokens': 2010, 'input_token_details': {'audio': 0, 'cache_read': 1792}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [46]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Los Angeles is currently warmer than San Francisco, with a temperature of 14.4°C (57.9°F) compared to San Francisco's 13.3°C (55.9°F).", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 2021, 'total_tokens': 2063, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1920}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-eb1fb3a3-0fdb-4db7-9d62-d7c42940d1ab-0', usage_metadata={'input_tokens': 2021, 'output_tokens': 42, 'total_tokens': 2063, 'input_token_details': {'audio': 0, 'cache_read': 1920}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [47]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please specify the two items, locations, or entities you would like me to compare in terms of warmth?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 182, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-7559adb9-8b5d-4890-afd4-ac694bca1dfb-0', usage_metadata={'input_tokens': 182, 'output_tokens': 25, 'total_tokens': 207, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


## Streaming tokens

In [65]:
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver

In [69]:
async with AsyncSqliteSaver.from_conn_string(":memory:") as memory:
        abot = Agent(model, [tool], system=prompt, checkpointer=memory)
        messages = [HumanMessage(content="What is the weather in SF?")]
        thread = {"configurable": {"thread_id": "4"}}
        async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
            kind = event["event"]
            print(event)
            if kind == "on_chat_model_stream":
                content = event["data"]["chunk"].content
                if content:
                    # Empty content in the context of OpenAI means
                    # that the model is asking for a tool to be invoked.
                    # So we only print non-empty content
                    print(content, end="|")

{'event': 'on_chain_start', 'run_id': 'a892061e-8ae8-4b5b-8fab-f98861de83a7', 'name': 'LangGraph', 'tags': [], 'metadata': {'thread_id': '4'}, 'data': {'input': {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={})]}}, 'parent_ids': []}
{'event': 'on_chain_start', 'name': '__start__', 'run_id': 'eeef4fcc-cf18-4c95-968c-7c06a775df4b', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'thread_id': '4', 'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_path': ('__pregel_pull', '__start__'), 'langgraph_checkpoint_ns': '__start__:c7d64b79-1eb7-9d40-1798-278330ba1cf4'}, 'data': {'input': {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={})]}}, 'parent_ids': []}
{'event': 'on_chain_start', 'name': '_write', 'run_id': '53c85fc4-83d7-4179-afb3-0a7ec3962f32', 'tags': ['seq:step:1', 'langsmith:hidden', 'langsmith:hidden'], 'metad